# LAND USE 01: LAND USE MAP GENERATOR
Note: this notebook generates 17 high-density plotly maps.
It runs pretty slowly.

# 01) IMPORT RAW DATA

In [1]:
import pandas as pd

In [2]:
# lookup table: county codes (from http://bit.ly/F7XW4J1JDataset0)
strCounty = 'F7XW4J1J (Anthropogenic Influences)/Dataset0_Boundary_IDs/boundary_ids-counties.txt'
# lookup table: state FIPS (from https://en.wikipedia.org/wiki/Federal_Information_Processing_Standard_state_code)
strFIPS = 'F7XW4J1J (Anthropogenic Influences)/StateFIPS.csv'
# lookup table: NAWQA Wall-to-Wall Anthropogenic Landuse Trends (NWALT) fields (from http://bit.ly/F7XW4J1JDataset8)
strNWALT = 'F7XW4J1J (Anthropogenic Influences)/NWALT.csv'

In [3]:
# import lookup tables
dfCounty = pd.read_csv(strCounty)
dfFIPS = pd.read_csv(strFIPS)
dfNWALT = pd.read_csv(strNWALT)

In [4]:
# examine County lookup dataframe
dfCounty.head()

,uniqid_co,GEOID10,NAMELSAD10,areakm2
0,1,4019,Pima County,23799.8
1,2,4017,Navajo County,25795.6
2,3,4012,La Paz County,11690.5
3,4,4007,Gila County,12420.2
4,5,4027,Yuma County,14294.4


In [5]:
# examine State lookup dataframe
dfFIPS.head()

,State,St,FIPS
0,Alabama,AL,1
1,Alaska,AK,2
2,American Samoa,AS,60
3,American Samoa,NaN,3
4,Arizona,AZ,4


In [6]:
# examine NWALT lookup dataframe
dfNWALT.head()

,Class,Defn,Definition
0,11,Water,Water
1,12,Wetlands,Core Wetlands
2,21,Dev-Trans,"Developed, Transportation: Major roads, rail a..."
3,22,Dev-CommSvcs,"Developed, Commercial/Services"
4,23,Dev-IndMil,"Developed, Industrial/Military"


In [7]:
# define paths to source files (from http://bit.ly/F7XW4J1JDataset8)
strLand1982 = 'F7XW4J1J (Anthropogenic Influences)/Dataset8_LandUse/landuse-county-nwalt1982.txt'
strLand1992 = 'F7XW4J1J (Anthropogenic Influences)/Dataset8_LandUse/landuse-county-nwalt1992.txt'
strLand2002 = 'F7XW4J1J (Anthropogenic Influences)/Dataset8_LandUse/landuse-county-nwalt2002.txt'
strLand2012 = 'F7XW4J1J (Anthropogenic Influences)/Dataset8_LandUse/landuse-county-nwalt2012.txt'

In [8]:
# import data
dfLand1982 = pd.read_csv(strLand1982)
dfLand1992 = pd.read_csv(strLand1992)
dfLand2002 = pd.read_csv(strLand2002)
dfLand2012 = pd.read_csv(strLand2012)

In [9]:
# examine sample dataframe
dfLand2012.head()

,uniqid_county,NWALT12_11_county,NWALT12_12_county,NWALT12_21_county,NWALT12_22_county,NWALT12_23_county,NWALT12_24_county,NWALT12_25_county,NWALT12_26_county,NWALT12_27_county,...,NWALT12_32_county,NWALT12_33_county,NWALT12_41_county,NWALT12_43_county,NWALT12_44_county,NWALT12_45_county,NWALT12_46_county,NWALT12_50_county,NWALT12_60_county,NWALT12_9999_county
0,1,0.03,0.01,0.23,0.41,0.16,0.21,0.75,1.27,0.19,...,3.35,0.02,0.52,0.48,0.02,0.00,1.56,75.77,14.41,0.03
1,2,0.06,0.01,0.18,0.05,0.01,0.02,0.01,0.17,0.07,...,0.84,0.02,0.03,0.06,0.10,0.01,10.73,87.19,0.44,0.00
2,3,0.28,0.03,0.24,0.03,0.00,0.01,0.03,0.07,0.08,...,0.38,0.00,0.00,0.56,4.11,0.10,3.11,80.14,10.81,0.00
3,4,0.67,0.01,0.18,0.04,0.01,0.03,0.03,0.19,0.07,...,0.57,0.00,0.25,0.01,0.01,0.00,0.01,89.52,8.38,0.00
4,5,0.15,0.02,0.22,0.19,0.12,0.05,0.32,0.23,0.11,...,0.49,0.01,0.01,5.04,0.75,0.02,0.19,65.57,26.35,0.06


# 02) REFORMAT DATA

In [10]:
# County dataframe: create new col to capture 5-digit state & county GEOID
dfCounty['GEOID5'] = ''
dfCounty['GEOID5'] = dfCounty['GEOID10'].astype('object')
# prepend four-digit GEOID5s with a zero
dfCounty['GEOID5'] = dfCounty['GEOID5'].apply(lambda x: str(x).zfill(5))
dfCounty['FIPS'] = dfCounty['GEOID5'].apply(lambda x: str(x)[:2])
# drop GEOID10
dfCounty = dfCounty.drop('GEOID10', axis=1)

In [11]:
# FIPS dataframe: format FIPS column as two-digit string
dfFIPS['FIPS'] = dfFIPS['FIPS'].astype('str')
for index, var in enumerate(dfFIPS['FIPS']):
    if float(var) < 10:
        dfFIPS.loc[index, 'FIPS'] = '0' + str(var)

In [12]:
# NWALT dataframe: ensure Classes are strings
dfNWALT['Class'] = dfNWALT['Class'].astype('str')

In [13]:
# define function to rename Land Use cols
def RenameCols(df, strYr):
    dictColNm = {'uniqid_county':'uniqid_co'}
    for index, row in dfNWALT.iterrows():
        dictColNm['NWALT' + strYr + '_' + dfNWALT.loc[index,'Class'] + '_county'] = \
            strYr + dfNWALT.loc[index,'Defn']
    df = df.rename(columns=dictColNm)
    return df

In [14]:
# call Rename Columns() to give dataframes English names
dfLand1982 = RenameCols(dfLand1982, '82')
dfLand1992 = RenameCols(dfLand1992, '92')
dfLand2002 = RenameCols(dfLand2002, '02')
dfLand2012 = RenameCols(dfLand2012, '12')

# 03) CREATE MASTER DATAFRAME & COMPUTE DIFFS

In [15]:
# create master dataframe dfLand
dfLand = pd.merge(dfLand1982, dfLand1992, on='uniqid_co')
dfLand = pd.merge(dfLand, dfLand2002, on='uniqid_co')
dfLand = pd.merge(dfLand, dfLand2012, on='uniqid_co')
dfLand = pd.merge(dfLand, dfCounty, on='uniqid_co')
dfLand = pd.merge(dfLand, dfFIPS, on='FIPS')

In [16]:
# define function to compute percentage-point change in Land Use, 1982-2012
def Compute82To12Chg(df, strCol):
    df['Chg-'+strCol] = df['12'+strCol] - df['82'+strCol]
    return

In [17]:
# create percentage-point change in Land Use cols
for row in dfNWALT.iterrows():
    Compute82To12Chg(dfLand, row[1][1])

In [18]:
# create Change dataframe
dfLandChg = dfLand.filter(regex="Chg|GEOID5")

In [19]:
# export Change dataframe
dfLandChg.to_csv('F7XW4J1J (Anthropogenic Influences)/Land Use Change, 1982-2012.csv', index=False, header=True)

# 04) MAP DIFFS BY COUNTY

In [20]:
# import plotly and set credentials
from PlotlyConfig import un, pkey 
import plotly
import plotly.plotly as py
import plotly.figure_factory as ff
import numpy as np
plotly.tools.set_credentials_file(username=un, api_key=pkey)

In [22]:
def MakeCtyFig(strVal, fRound=False, fBin=True, strPrefix='Chg-', strFIPS='GEOID5', df=dfLandChg):
    # create lists of parameter values
    lstFIPS = df[strFIPS].tolist()
    lstVal = df[strPrefix + strVal].tolist()
    if fRound:
        lstVal = [round(val, 2) for val in lstVal]
    # lstColor = ['#FF0000','#FF8000','#FFFF00','#80FF00','#00FF00','#00FF80','#0080FF','#0000FF'] 8-color ROYGB
    lstColor = ['#FF0040','#FF0000','#FF2800','#FF5000', '#FF7800','#FFa000','#FFc800','#FFf000', \
                '#b0ff00','#17ff00','#00ff83','#00e4ff', '#00a4ff','#0064ff','#0022ff','#0100ff', \
                '#0500ff'] # 13-color ROYGB
    intColor = len(lstColor)
    intBinSize = 1 / (intColor + 1) * 100
    lstBin = list(np.linspace(np.percentile(lstVal, intBinSize), \
                              np.percentile(lstVal, 100 - intBinSize), \
                              intColor - 1))
    # set fig variable
    if fBin:
        fig = ff.create_choropleth(
            fips=lstFIPS, values=lstVal,
            binning_endpoints=lstBin,
            colorscale=lstColor,
            show_state_data=False,
            show_hover=True, centroid_marker={'opacity': 0},
            asp=2.9, title='Percentage Point Change in ' + \
                dfNWALT.loc[dfNWALT['Defn']==strVal, 'Definition'].iloc[0] + \
                ', 1982-2012',
            legend_title='Percentage Point Change'
        )
    else:
        fig = ff.create_choropleth(
            fips=lstFIPS, values=lstVal,
            title='Percentage Point Change in ' + \
                dfNWALT.loc[dfNWALT['Defn']==strVal, 'Definition'].iloc[0] + \
                ', 1982-2012',
            legend_title='Percentage Point Change'
        )
        
    return fig

In [23]:
# no data for Wetlands, VeryLowUse, or Unknown

In [24]:
py.iplot(MakeCtyFig('Water', True), filename='choropleth_full_usa')

C:\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




The draw time for this plot will be slow for clients without much RAM.


C:\Anaconda3\lib\site-packages\chart_studio\api\v1\clientresp.py:44: UserWarning:

Estimated Draw Time Slow

C:\Anaconda3\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [23]:
py.iplot(MakeCtyFig('Dev-Trans', True), filename='choropleth_full_usa')

The draw time for this plot will be slow for clients without much RAM.


In [24]:
py.iplot(MakeCtyFig('Dev-CommSvcs', True), filename='choropleth_full_usa')

The draw time for this plot will be slow for clients without much RAM.


In [25]:
py.iplot(MakeCtyFig('Dev-IndMil', True), filename='choropleth_full_usa')

The draw time for this plot will be slow for clients without much RAM.


In [26]:
py.iplot(MakeCtyFig('Dev-Recr', True), filename='choropleth_full_usa')

The draw time for this plot will be slow for clients without much RAM.


In [27]:
py.iplot(MakeCtyFig('Dev-ResHi', True), filename='choropleth_full_usa')

The draw time for this plot will be slow for clients without much RAM.


In [28]:
py.iplot(MakeCtyFig('Dev-ResLoMed', True), filename='choropleth_full_usa')

The draw time for this plot will be slow for clients without much RAM.


In [29]:
py.iplot(MakeCtyFig('Dev-Other', True), filename='choropleth_full_usa')

The draw time for this plot will be slow for clients without much RAM.


In [30]:
py.iplot(MakeCtyFig('SemiDev-UrbIntHi', True), filename='choropleth_full_usa')

The draw time for this plot will be slow for clients without much RAM.


In [31]:
py.iplot(MakeCtyFig('SemiDev-UrbIntLoMed', True), filename='choropleth_full_usa')

The draw time for this plot will be slow for clients without much RAM.


In [32]:
py.iplot(MakeCtyFig('SemiDev-Other', True), filename='choropleth_full_usa')

The draw time for this plot will be slow for all clients.


C:\Anaconda3\lib\site-packages\chart_studio\api\v1\clientresp.py:44: UserWarning:

Estimated Draw Time Too Long



In [33]:
py.iplot(MakeCtyFig('Mining', True), filename='choropleth_full_usa')

The draw time for this plot will be slow for clients without much RAM.


In [34]:
py.iplot(MakeCtyFig('Crops', True), filename='choropleth_full_usa')

The draw time for this plot will be slow for clients without much RAM.


In [35]:
py.iplot(MakeCtyFig('Pasture', True), filename='choropleth_full_usa')

The draw time for this plot will be slow for clients without much RAM.


In [36]:
py.iplot(MakeCtyFig('Grazing1', True), filename='choropleth_full_usa')

The draw time for this plot will be slow for clients without much RAM.


In [37]:
py.iplot(MakeCtyFig('Grazing2', True), filename='choropleth_full_usa')

The draw time for this plot will be slow for clients without much RAM.


In [38]:
py.iplot(MakeCtyFig('LowUse', True), filename='choropleth_full_usa')

The draw time for this plot will be slow for clients without much RAM.
